<a href="https://colab.research.google.com/github/ankitstar01/DeepLearning/blob/master/Stock_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir Data
!mkdir Data/Historic
!mkdir Data/Recent
!mkdir Models

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
import os
 
from sklearn.preprocessing import MinMaxScaler
from polygon import RESTClient
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Activation
 
API_KEY = 'vqeohd8x5k1x6PKiisJQS8RgS3A1iRQC'
ticker = 'GME'

ModuleNotFoundError: ignored

In [ ]:
f = open('Data/Historic/' + ticker + '.csv', 'w')
with RESTClient(API_KEY) as client:
    resp = client.stocks_equities_aggregates(ticker=ticker, multiplier=15, timespan='minute',
                                              from_='2020-10-03', to='2021-02-03', unadjusted=False,
                                              sort='asc', limit=50000)
    resp = resp.results
    f.write('timestamp,open,low,high,volume,close\n')
    for query in resp:
        f.write(str(query['t']) + ',' + str(query['o']) + ',' + str(query['l']) + ','
                + str(query['h']) + ',' + str(query['v']) + ',' + str(query['c']) + '\n')
f.close()
f = open('Data/Historic/' + ticker + '.csv', 'r')
df = pd.read_csv(f)

In [ ]:
df.tail()

In [ ]:
df = df.drop(['timestamp'], axis=1)
df.tail()

,open,low,high,volume,close
4582,90.05,89.50,90.09,24463.0,89.6000
4583,89.30,89.01,89.50,28862.0,89.0701
4584,89.20,88.50,89.20,38626.0,89.0000
4585,89.00,87.50,89.00,56526.0,87.5400
4586,87.50,87.37,89.35,61108.0,88.8800


In [ ]:
rows = len(df)
df_training = df[:rows - 16]
df_testing = df[-16:]
training_df = df_training.copy()

scaler = MinMaxScaler()
df_training = scaler.fit_transform(df_training)
df_training.shape

(4571, 5)

In [ ]:
x_train, y_train, x_test, y_test = [], [], [], []
for i in range(15, df_training.shape[0]):
    x_train.append(df_training[i - 15: i])
    y_train.append(df_training[i, 0])
x_train, y_train = np.asarray(x_train), np.asarray(y_train)
print(x_train.shape)
print(y_train.shape)

(4556, 15, 5)
(4556,)


In [ ]:
df_testing = scaler.transform(df_testing)
x_test = [df_testing[:-1]]
x_test = np.asarray(x_test)
y_test = []
y_test.append(df_testing[-1, 0])
y_test = np.asarray(y_test)
print(x_test.shape)
print(y_test.shape)

(1, 15, 5)
(1,)


In [ ]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(15, 5), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=100, activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=120, activation='relu', return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
143/143 [==============================] - 14s 63ms/step - loss: 1.9601e-04
Epoch 2/20
143/143 [==============================] - 9s 62ms/step - loss: 2.5408e-04
Epoch 3/20
143/143 [==============================] - 9s 62ms/step - loss: 2.1044e-04
Epoch 4/20
143/143 [==============================] - 9s 62ms/step - loss: 1.9300e-04
Epoch 5/20
143/143 [==============================] - 9s 64ms/step - loss: 1.8986e-04
Epoch 6/20
143/143 [==============================] - 9s 62ms/step - loss: 2.0630e-04
Epoch 7/20
143/143 [==============================] - 9s 61ms/step - loss: 2.1846e-04
Epoch 8/20
143/143 [==============================] - 9s 62ms/step - loss: 2.6639e-04
Epoch 9/20
143/143 [==============================] - 9s 61ms/step - loss: 1.7130e-04
Epoch 10/20
143/143 [==============================] - 9s 63ms/step - loss: 2.0389e-04
Epoch 11/20
143/143 [==============================] - 9s 62ms/step - loss: 2.1563e-04
Epoch 12/20
143/143 [==============================

In [ ]:
scale = 1/scaler.scale_[0]
prediction = model.predict(x_test)
prediction = prediction * scale
print(prediction[0][0])
print((y_test * scale)[0])

79.08253
78.38


In [ ]:
model.save('Models/' + ticker)

INFO:tensorflow:Assets written to: Models/GME/assets


In [ ]:
model = keras.models.load_model('Models/' + ticker)

In [ ]:
f = open('Data/Recent/' + ticker + '.csv', 'w')
with RESTClient(API_KEY) as client:
    resp = client.stocks_equities_aggregates(ticker=ticker, multiplier=15, timespan='minute',
                                             from_='2021-02-04', to='2021-02-06', unadjusted=False,
                                             sort='asc', limit=50000)
    resp = resp.results
    f.write('open,low,high,volume,close\n')
    for query in resp:
        f.write(str(query['o']) + ',' + str(query['l']) + ','
                + str(query['h']) + ',' + str(query['v']) + ',' + str(query['c']) + '\n')
f.close()
f = open('Data/Recent/' + ticker + '.csv', 'r')
df = pd.read_csv(f)

In [ ]:
df.tail()

,open,low,high,volume,close
123,68.9700,66.50,69.00,56202.0,67.20
124,67.2000,66.60,67.52,32799.0,66.65
125,66.5000,66.50,67.45,42314.0,67.45
126,67.4499,66.89,68.00,44966.0,66.99
127,66.8800,66.46,67.11,99084.0,66.46


In [ ]:
scaler = MinMaxScaler()
df = scaler.fit_transform(df)

In [ ]:
testing_df = [df[-16:-1]]
testing_df = np.asarray(testing_df)
prediction = model.predict(testing_df)
prediction = prediction * (1 / scaler.scale_[0])
answer = df[-1][0] * (1 / scaler.scale_[0])

It shows prediction is 16 and actual is 17, but in the table it says the close price is $67. I am not sure what this issue is. The same is for the prediction above except its different values



In [ ]:
print('Prediction:', prediction[0][0])
print('Actual:', answer)

Prediction: 16.054567
Actual: 17.129999999999995
